In [1]:
import cobra
import numpy as np
from cobra import Model, Reaction, Metabolite
import matplotlib.pyplot as plt
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis

# Function to set glycerol minimal medium

In [4]:
def set_glycerol_minimal_medium(model):
    
    # set lower bound of all exchange reactions to zero
    for rxn in model.exchanges:
        rxn.lower_bound = 0

    # turn on specific fluxes
    minimal__media = {
            'EX_glyc_e_': -10,       # glycerol
            'EX_nh4_e_': -1000,      # ammonium
            'EX_pi_e_':  -1000,      # phosphate
            'EX_so4_e_': -1000,      # sulfate
            'EX_fe3_e_': -1000,      # fe3+
            'EX_mn2_e_': -1000,      # mn2+
            'EX_zn2_e_': -1000,      # zn2+
            'EX_cu2_e_': -1000,      # cu2+
            'EX_ca2_e_': -1000,      # ca2+
            'EX_cl_e_' : -1000,      # cl-
            'EX_cobalt2_e_': -1000,  # cobalt
            'EX_k_e_':   -1000,      # k
            'EX_mobd_e_'   : -1000,  # mo6+
            'EX_na1_e_'    : -1000,     # na+
            'EX_ni2_e_'    : -1000,     # ni2+
            'EX_mg2_e_': -1000,      # mg2+
            'EX_o2_e_':  -1000
        }

    for r in minimal__media.keys():
        try:
            rxn = model.reactions.get_by_id(r)
            rxn.lower_bound = minimal__media[r]
            rxn.upper_bound = 1000
        except:
            print("The reaction {} does not exist in model.".format(str(r)))
            continue
            
    return model

# Tune flux through reaction AKGDH and simulate overflow

## carbon is limiting

In [37]:
model = cobra.io.load_json_model('../add_RL_production/SI3_iJN1411final_flux_w_rhamnolipid_biosynthesis.json')
model.solver = 'cplex'
model = set_glycerol_minimal_medium(model)

exchange_flux_max = pd.DataFrame()
exchange_flux_min = pd.DataFrame()
max_growth_flux = []
AKGDH_flux =  np.arange(0, 2, 1)
for f in AKGDH_flux:
    print(f)
    model.reactions.AKGDH.lb = f
    model.reactions.AKGDH.ub = f
    max_growth = model.slim_optimize()
    if str(max_growth) == 'nan':
        break
    max_growth_flux.append(max_growth)
    
    # flux variability analysis
    fva = cobra.flux_analysis.flux_variability_analysis(model, model.exchanges, loopless=True, fraction_of_optimum=1.00)
    if f==0:
        exchange_flux_max = fva['maximum'].to_frame()
        exchange_flux_min = fva['minimum'].to_frame()
    else:
        exchange_flux_max = pd.merge(exchange_flux_max, fva['maximum'].to_frame(), left_index=True, right_index=True)
        exchange_flux_min = pd.merge(exchange_flux_min, fva['minimum'].to_frame(), left_index=True, right_index=True)
        
exchange_flux_max.columns = [str(f) for f in AKGDH_flux]
exchange_flux_min.columns = [str(f) for f in AKGDH_flux]

0
1


### plot

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=)